In [86]:
import os
import pandas as pd
import numpy as np
import re
import math
import sys

import sys
sys.path.insert(0, '../') # so we can access our config files
from src.data.youtubeDownloader import Downloader

In [90]:
homepage = pd.read_csv('../data/audit/youtube/raw/16780532672002707-traditional-young-watch_ratio0.5-homepage_recs.csv')
side = pd.read_csv('../data/audit/youtube/raw/1678002204369677-traditional-young-watch_time1-video_recs.csv')

In [91]:
homepage.head()

,url,recommendation_source,videos_watched,Start Time,Age,Finance Video Type
0,https://www.youtube.com/watch?v=KkEuUQN5t58,homepage,18,1.678053e+09,young,traditional
1,https://www.youtube.com/watch?v=Plq6DtKYrk4,homepage,18,1.678053e+09,young,traditional
2,https://www.youtube.com/watch?v=pEqITNKwO04,homepage,18,1.678053e+09,young,traditional
3,https://www.youtube.com/watch?v=LjJGpdaWbD4,homepage,18,1.678053e+09,young,traditional
4,https://www.youtube.com/watch?v=cOoSQT4i0yI,homepage,18,1.678053e+09,young,traditional


In [98]:
homepage.iloc[90]

url                      https://www.youtube.com/watch?v=87qL_bZnAMk
recommendation_source                                       homepage
videos_watched                                                    44
Start Time                                         1678053267.200271
Age                                                            young
Finance Video Type                                       traditional
Name: 90, dtype: object

In [4]:
side.head()

,url,reccommendation_source,videos_watched,Start Time,Age,Finance Video Type
0,https://www.youtube.com/watch?v=gFQNPmLKj1k,https://www.youtube.com/watch?v=W3CDMcWlPyM,1,1.678002e+09,young,traditional
1,https://www.youtube.com/watch?v=qc4GU_wHdqw,https://www.youtube.com/watch?v=W3CDMcWlPyM,1,1.678002e+09,young,traditional
2,https://www.youtube.com/watch?v=qlf9-573MhI,https://www.youtube.com/watch?v=W3CDMcWlPyM,1,1.678002e+09,young,traditional
3,https://www.youtube.com/watch?v=9AThycGCakk,https://www.youtube.com/watch?v=W3CDMcWlPyM,1,1.678002e+09,young,traditional
4,https://www.youtube.com/watch?v=iONDebHX9qk,https://www.youtube.com/watch?v=W3CDMcWlPyM,1,1.678002e+09,young,traditional


In [85]:
homepage['url'].apply(extract_video_id)

0     Vus3x9fQDKE
1     9be3UJvjpZw
2     eY8lhv-Uzdc
3     TwvJGHI8EO0
4     7A7lGm9-kZc
         ...     
75    RlKxgwMzVdA
76    vfaa_nonqb8
77    pgS0EhJDri0
78    v96lDsnJIxY
79    dP95z1QgnXk
Name: url, Length: 80, dtype: object

In [80]:
(480 * 5) / 60

40.0

In [30]:
def extract_video_id(url):
    try: 
        # Search for the video id
        match = re.search(r'[?&]v=([^&]*)?', url)

        # If we don't find the video id, try and use a normal split
        if match:
            videoId = match.group(1).split('&')[0]
        else:
            if 'v=' in url:
                videoId = url.split('v=')
                if len(videoId) > 1:
                    videoId = url.split('v=')[1]
                else:
                    videoId = None
            elif 'shorts/' in url:
                videoId = url.split('shorts/')
                if len(videoId) > 1:
                    videoId = url.split('shorts/')[1]
                else:
                    videoId = None
            else:
                videoId = None
    except:
        return None
    return videoId

In [83]:
homepage['url'].apply(extract_video_id)

0     Vus3x9fQDKE
1     9be3UJvjpZw
2     eY8lhv-Uzdc
3     TwvJGHI8EO0
4     7A7lGm9-kZc
         ...     
75    RlKxgwMzVdA
76    vfaa_nonqb8
77    pgS0EhJDri0
78    v96lDsnJIxY
79    dP95z1QgnXk
Name: url, Length: 80, dtype: object

In [75]:
def download_audit_videos(self, video_ids, load_path, save_path):
        
    # store our downloaded videos
    videos = []
    df = pd.DataFrame()
    
    # check for progress
    saved_files = os.listdir('../data/audit/youtube/processed/')
    downloader = Downloader()b
    
    print("Starting download...")
    logging.info("Starting download...")
    start = time.time()

    if not downloader.apiObjectExists():
        print("API object not found")
        logging.info("API object not found")
        return pd.DataFrame()
    
    if 'audit_progress.csv' in saved_files:
        
        print('Progress found, starting from where we left off.')
        logging.info('Progress found, starting from where we left off.')
        progress = pd.read_csv('../data/audit/youtube/processed/' + 'audit_progress.csv')
        
        progress_video_ids = list(progress['video_id'])
        
        video_ids = [video_id for video_id in video_ids if video_id not in progress_video_ids]
        
        df = pd.concat([df, progress])
        
    # Collect our data, Added TQDM progress bar
    for i in tqdm(range(len(video_ids))):

        try:
            video_id = video_ids[i]
            label = labels[i]

            logging.info("Downloading " + video_id)

            downloader.setVideoId(video_id)

            videoMetaData = downloader.getVideoMetadata()
            time.sleep(random.randint(1, 5))
            videoTranscript = downloader.getVideoTranscript()

            video = {
                "video_id": video_id,
                "title": videoMetaData["title"],
                "description": videoMetaData["description"],
                "tags": videoMetaData["tags"],
                "raw_duration": videoMetaData["raw_duration"],
                "duration": videoMetaData["duration"],
                "cleaned_transcript": videoTranscript["cleaned_transcript"],
                "raw_transcript": videoTranscript["raw_transcript"], 
                "link": "www.youtube.com/watch?v=" + video_id,
            }

            videos.append(video)

            logging.info("Finished downloading " + video_id)
        except:
            # If we run into any error
            print("Ran into exception, saving progress and stopping.")
            logging.info("Ran into exception, saving progress and stopping.")

            df = pd.concat([df, pd.DataFrame(videos)])

            df.to_csv('../data/audit/youtube/processed/' + 'audit_progress.csv', index_label=False)

            print("Saved file to:  {0}".format('../data/audit/youtube/processed/' + 'audit_progress.csv'))
            logging.info("Saved file to:  {0}".format('../data/audit/youtube/processed/' + 'audit_progress.csv'))

            return df

    # If we complete the download.
    print("Downloaded {0} in {1} seconds.".format(len(targets), time.time() - start))
    logging.info("Downloaded {0} in {1} seconds.".format(len(targets), time.time() - start))

    df = pd.concat([df, pd.DataFrame(videos)])

    df.to_csv('../data/audit/youtube/processed/' + 'audit_progress.csv', index_label=False)

    print("Saved file to:  {0}".format('../data/audit/youtube/processed/' + 'audit_progress.csv'))
    logging.info("Saved file to:  {0}".format('../data/audit/youtube/processed/' + 'audit_progress.csv'))

    return df

In [40]:
downloader = Downloader()

Successfully built YouTube API object


In [104]:
pd.set_option('display.max_rows', 500)

In [105]:
homepage_downloads = pd.read_csv('../data/audit/youtube/processed/downloaded_homepage_traditional_young.csv')

In [106]:
sidebar_downloads = pd.read_csv('../data/audit/youtube/processed/downloaded_sidebar_traditional_young.csv')

In [108]:
homepage_downloads

,video_id,title,description,tags,raw_duration,duration,cleaned_transcript,raw_transcript,link
0,KkEuUQN5t58,how i make money as a college student // not a...,Follow up Q&A video here: https://youtu.be/SdF...,"['uchicago', 'college side hustle', 'how to ma...",PT14M1S,00:14:01,no youtube\nno social media following. no day...,[{'text': 'no youtube\nno social media followi...,www.youtube.com/watch?v=KkEuUQN5t58
1,Plq6DtKYrk4,Who Has The Highest SAT Score? | Ranking,Have you heard about nectar? Subscribe now! ht...,"['jubilee', 'jubilee media', 'jubilee project'...",PT12M31S,00:12:31,"(upbeat music) ""notification pop up sounds"" - ...","[{'text': '(upbeat music)', 'start': 0.738, 'd...",www.youtube.com/watch?v=Plq6DtKYrk4
2,pEqITNKwO04,COLLEGE DECISION REACTIONS | 16 YEARS OLD | HA...,"Hey everyone, here are my 2021 college decisio...","['College Decision Reactions', 'College', 'App...",PT25M15S,00:25:15,i am submitting my georgia tech. application ...,"[{'text': 'i am submitting my georgia tech', '...",www.youtube.com/watch?v=pEqITNKwO04
3,LjJGpdaWbD4,Become Aesthetic In 2023,Being aesthetic means having balanced symmetri...,"['Aesthetic', 'aesthetic training', 'gym aesth...",PT7M43S,00:07:43,being aesthetic is something I've always. wan...,"[{'text': ""being aesthetic is something I've a...",www.youtube.com/watch?v=LjJGpdaWbD4
4,cOoSQT4i0yI,YOU MUST TAKE BACK CONTROL OF YOUR LIFE AND GE...,NaN,"['#AbsoluteMotivation', '#MotivationalVideo', ...",PT8M2S,00:08:02,you don't have to have this victim. mentality...,"[{'text': ""you don't have to have this victim""...",www.youtube.com/watch?v=cOoSQT4i0yI
5,rL2SRBEDCQw,college move in day 2019! (freshman year),MOVING INTO COLLEGE AHHHHHH!!!!! ENJOY!!!!!\n\...,"['college move in day', 'college move', 'colle...",PT12M52S,00:12:52,this is so sad I leave for college today. so ...,[{'text': 'this is so sad I leave for college ...,www.youtube.com/watch?v=rL2SRBEDCQw
6,Phq3xkZOchE,I Followed The Average American Diet (How Bad ...,The United States Of America has battled with ...,"['nutrition', 'food', 'fast food', 'american d...",PT23M31S,00:23:31,siri how do i get big at home. oh that sounds...,"[{'text': 'siri how do i get big at home', 'st...",www.youtube.com/watch?v=Phq3xkZOchE
7,Xb1dCbiqiTU,[4K]Walk in Guizhou University and see the lif...,This is the West Campus of Guizhou University....,"['4K', '4Kchina', 'ChinaWalk', 'china', 'GuiYa...",PT57M42S,00:57:42,NaN,{},www.youtube.com/watch?v=Xb1dCbiqiTU
8,AVSaPwDPZhk,the flops of genius,100% inspired by witch bish\nwitch bish: https...,"['genius', 'genius flops', 'gabbie hanna', 'ga...",PT2M30S,00:02:30,NaN,{},www.youtube.com/watch?v=AVSaPwDPZhk
9,MMmOLN5zBLY,The benefits of a bilingual brain - Mia Nacamulli,Check out our Patreon page: https://www.patreo...,"['Mia Nacamulli', 'language', 'speaking langua...",PT5M4S,00:05:04,¿Hablas español? Parlez-vous français? \n你会说中文...,[{'text': '¿Hablas español? Parlez-vous frança...,www.youtube.com/watch?v=MMmOLN5zBLY
